In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [2]:
# Check if MPS is available
device = "mps" if torch.backends.mps.is_available() else "cpu"

# Check if CUDA is available
# device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/deepseek-coder-1.3b-instruct",
    trust_remote_code=True,
    torch_dtype=torch.float16  # MPS prefers float16 instead of bfloat16
).to(device)

# If you are using CUDA, you can use bfloat16 instead of float16
# model = AutoModelForCausalLM.from_pretrained(
#     "deepseek-ai/deepseek-coder-1.3b-instruct",
#     trust_remote_code=True,
#     torch_dtype=torch.bfloat16
# ).to(device)

print("Running on:", device)

Running on: mps


In [4]:
from sentence_transformers import SentenceTransformer, util

In [5]:
# Load a sentence similarity model
similarity_model = SentenceTransformer('sentence-transformers/gtr-t5-large')

In [6]:
def generate_code_description(messages):
    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
    # tokenizer.eos_token_id is the id of <|EOT|> token
    outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
    description = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
    return description

In [7]:
def compare_descriptions(generated_desc, user_desc):
    embeddings = similarity_model.encode([generated_desc, user_desc], convert_to_tensor=True)
    similarity_score = util.pytorch_cos_sim(embeddings[0], embeddings[1]).item()
    return similarity_score

In [8]:
def initial_code_verification(messages, user_description, threshold=0.8):
    generated_description = generate_code_description(messages)
    similarity_score = compare_descriptions(generated_description, user_description)

    result = {
        "generated_description": generated_description,
        "similarity_score": similarity_score,
        "matches_expectation": similarity_score >= threshold
    }

    return result

In [9]:
def read_code_from_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read()

#### Checking for Python Code

In [10]:
prefix = "Write a short summary for the following code: "

file_path = "Code-Files/factorial.py"
code_snippet = read_code_from_file(file_path)

In [11]:
messages=[
    { 'role': 'user', 'content': prefix+code_snippet}
]

In [12]:
user_description = "This function calculates the factorial of a given number recursively."

In [13]:
result = initial_code_verification(messages, user_description)

/opt/miniconda3/envs/malcodeai/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [14]:
print("User Description:", user_description)

User Description: This function calculates the factorial of a given number recursively.


In [15]:
print("Generated Description:", result["generated_description"])

Generated Description: This Python function calculates the factorial of a given number `n`. The factorial of a number is the product of all positive integers less than or equal to that number. For example, the factorial of 5 is 5*4*3*2*1 = 120.

The function uses recursion to calculate the factorial. It starts by checking if `n` is 0, in which case it returns 1 (since the factorial of 0 is 1). If `n` is not 0, the function calls itself with `n-1` and multiplies the result by `n`. This continues until `n` is 0, at which point it returns 1. The result is the factorial of the original number.



In [16]:
print("Similarity Score:", result["similarity_score"])

Similarity Score: 0.8649032115936279


In [17]:
print("Matches Expectation:", result["matches_expectation"])

Matches Expectation: True


#### Checking for Java Code

In [ ]:
prefix = "Write a short summary for the following code: "

file_path = "Code-Files/FactorialRecursive.java"
code_snippet = read_code_from_file(file_path)

In [19]:
messages=[
    { 'role': 'user', 'content': prefix+code_snippet}
]

In [20]:
user_description = "The FactorialRecursive class calculates factorials of non-negative integers, demonstrating both int (prone to overflow) and BigInteger (handles large numbers) recursive approaches.  factorial(int n) calculates factorials for smaller integers, while factorialBig(BigInteger n) uses BigInteger to prevent overflow for larger numbers. The main method showcases both, highlighting the overflow issue with int and the correct results with BigInteger."

In [21]:
result = initial_code_verification(messages, user_description)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


In [22]:
print("User Description:", user_description)

User Description: The FactorialRecursive class calculates factorials of non-negative integers, demonstrating both int (prone to overflow) and BigInteger (handles large numbers) recursive approaches.  factorial(int n) calculates factorials for smaller integers, while factorialBig(BigInteger n) uses BigInteger to prevent overflow for larger numbers. The main method showcases both, highlighting the overflow issue with int and the correct results with BigInteger.


In [23]:
print("Generated Description:", result["generated_description"])

Generated Description: This code is a simple implementation of a recursive function to calculate the factorial of a number. The factorial of a number is the product of all positive integers less than or equal to that number. For example, the factorial of 5 (5!) is 5*4*3*2*1 = 120.

The `factorial` method is a recursive function that calculates the factorial of a given number. It uses a base case to handle the case where the number is 0, in which case it returns 1. For all other numbers, it recursively calls itself with the number decremented by 1 and multiplies the result with the current number.

The `main` method demonstrates the use of the `factorial` method with different inputs to illustrate its functionality. It also demonstrates how the method can handle large numbers and potential overflow issues.

The `factorialBig` method is a more advanced version of the `factorial` method that uses the `BigInteger` class to handle larger numbers and avoid overflow. It uses the same recursiv

In [24]:
print("Similarity Score:", result["similarity_score"])

Similarity Score: 0.9296317100524902


In [25]:
print("Matches Expectation:", result["matches_expectation"])

Matches Expectation: True


#### After checking with both Python and Java, we can notice that the model works well to generate descriptions of files having code in various problems.

#### DeepSeek Coder: Pretrained on 2 Trillion tokens over more than 80 programming languages.

In [26]:
wrong_user_description = "The BinarySearch class implements a binary search algorithm to efficiently find a target value within a sorted array of integers. The binarySearch(int[] arr, int target) method takes the sorted array and the target value as input, returning the index of the target if found, and -1 otherwise. It repeatedly divides the search interval in half, eliminating the portion that cannot contain the target."

In [27]:
result = initial_code_verification(messages, wrong_user_description)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


In [28]:
print("User Description:", wrong_user_description)

User Description: The BinarySearch class implements a binary search algorithm to efficiently find a target value within a sorted array of integers. The binarySearch(int[] arr, int target) method takes the sorted array and the target value as input, returning the index of the target if found, and -1 otherwise. It repeatedly divides the search interval in half, eliminating the portion that cannot contain the target.


In [29]:
print("Generated Description:", result["generated_description"])

Generated Description: This code is a simple implementation of a recursive function to calculate the factorial of a number. The factorial of a number is the product of all positive integers less than or equal to that number. For example, the factorial of 5 (5!) is 5*4*3*2*1 = 120.

The `factorial` method is a recursive function that calculates the factorial of a given number. It uses a base case to handle the case where the number is 0, in which case it returns 1. For all other numbers, it recursively calls itself with the number decremented by 1 and multiplies the result with the current number.

The `main` method demonstrates the use of the `factorial` method with different inputs to illustrate its functionality. It also demonstrates how the method can handle large numbers and potential overflow issues.

The `factorialBig` method is a more advanced version of the `factorial` method that uses the `BigInteger` class to handle larger numbers and avoid overflow. It uses the same recursiv

In [30]:
print("Similarity Score:", result["similarity_score"])

Similarity Score: 0.6844414472579956


In [31]:
print("Matches Expectation:", result["matches_expectation"])

Matches Expectation: False
